In [1]:
# Imports
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import DataLoader, TensorDataset

# Load dataset
df = pd.read_csv(r"C:\Users\Arhamsoft\Desktop\Talha Talib Thesis\german_credit_data.csv")
df.dropna(inplace=True)

# Show target values
print("Unique values in 'kredit':", df['kredit'].unique())

# Target and features
y = df['kredit']
X = df.drop(columns=['kredit'])

# Encode categorical features
for col in X.select_dtypes(include='object').columns:
    X[col] = LabelEncoder().fit_transform(X[col])

# Normalize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42, stratify=y
)

# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Dataloader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# FFNN Definition
class FFNN(nn.Module):
    def __init__(self, input_dim):
        super(FFNN, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 2)
        )
    
    def forward(self, x):
        return self.network(x)

# Initialize
model = FFNN(input_dim=X_train.shape[1])
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(100):
    model.train()
    for xb, yb in train_loader:
        optimizer.zero_grad()
        outputs = model(xb)
        loss = criterion(outputs, yb)
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
with torch.no_grad():
    logits = model(X_test_tensor)
    y_pred = torch.argmax(logits, axis=1).numpy()

# Performance metrics
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print results
print("\nFeedforward Neural Network Performance on German Credit:")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1 Score : {f1:.4f}")


Unique values in 'kredit': [1 0]

Feedforward Neural Network Performance on German Credit:
Accuracy : 0.6933
Precision: 0.7757
Recall   : 0.7905
F1 Score : 0.7830


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from fairlearn.metrics import (
    MetricFrame, 
    demographic_parity_difference, 
    equalized_odds_difference, 
    selection_rate
)
from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.preprocessing import Reweighing

# Load and clean dataset
df = pd.read_csv(r"C:\Users\Arhamsoft\Desktop\Talha Talib Thesis\german_credit_data.csv")
df.dropna(inplace=True)

# Define protected attribute
df['sex_binary'] = df['famges'].apply(lambda x: 0 if x == 4 else 1)

# Target and protected
y = df["kredit"]
protected = df["sex_binary"]

# Prepare features (exclude target and protected)
X = df.drop(columns=["kredit", "sex_binary"])

# Encode categorical features
X_encoded = X.copy()
for col in X_encoded.select_dtypes(include="object").columns:
    X_encoded[col] = LabelEncoder().fit_transform(X_encoded[col])

# Normalize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)
X_df = pd.DataFrame(X_scaled, columns=X_encoded.columns)

# Final combined DataFrame for AIF360
df_ffnn = pd.concat([
    X_df.reset_index(drop=True),
    y.reset_index(drop=True).rename("target"),
    protected.reset_index(drop=True).rename("sex_binary")
], axis=1)

# BinaryLabelDataset
bld = BinaryLabelDataset(
    df=df_ffnn,
    label_names=["target"],
    protected_attribute_names=["sex_binary"],
    favorable_label=1,
    unfavorable_label=0
)

# Split into train/test
train_bld, test_bld = bld.split([0.7], shuffle=True)

# Apply Reweighing
rw = Reweighing(privileged_groups=[{"sex_binary": 1}], unprivileged_groups=[{"sex_binary": 0}])
train_rw = rw.fit_transform(train_bld)

# Convert to tensors
X_train_rw = torch.tensor(train_rw.features, dtype=torch.float32)
y_train_rw = torch.tensor(train_rw.labels.ravel(), dtype=torch.long)
sample_weights = torch.tensor(train_rw.instance_weights, dtype=torch.float32)

train_dataset = TensorDataset(X_train_rw, y_train_rw, sample_weights)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

# FFNN
class FFNN(nn.Module):
    def __init__(self, input_dim):
        super(FFNN, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 2)
        )
    def forward(self, x):
        return self.network(x)

# Initialize model
model = FFNN(input_dim=X_train_rw.shape[1])
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(reduction='none')

# Training loop
for epoch in range(100):
    model.train()
    for xb, yb, wb in train_loader:
        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        (loss * wb).mean().backward()
        optimizer.step()

# Evaluate on test set
model.eval()
X_test_tensor = torch.tensor(test_bld.features, dtype=torch.float32)
y_test_array = test_bld.labels.ravel()

# Get protected attribute from test set
test_df, _ = test_bld.convert_to_dataframe()
prot_test_array = test_df["sex_binary"].values.astype(int)

with torch.no_grad():
    logits = model(X_test_tensor)
    y_pred = torch.argmax(logits, axis=1).numpy()

# Performance metrics
acc = accuracy_score(y_test_array, y_pred)
prec = precision_score(y_test_array, y_pred, zero_division=0)
rec = recall_score(y_test_array, y_pred, zero_division=0)
f1 = f1_score(y_test_array, y_pred, zero_division=0)

print("Reweighed FFNN Performance (German Credit):")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1 Score : {f1:.4f}")

# Fairness metrics
fair_metrics = MetricFrame(
    metrics={
        "accuracy": accuracy_score,
        "precision": precision_score,
        "recall": recall_score,
        "f1": f1_score,
        "selection_rate": selection_rate
    },
    y_true=y_test_array,
    y_pred=y_pred,
    sensitive_features=prot_test_array
)

spd = demographic_parity_difference(y_test_array, y_pred, sensitive_features=prot_test_array)
eod = equalized_odds_difference(y_test_array, y_pred, sensitive_features=prot_test_array)
di_ratio = fair_metrics.by_group['selection_rate'].max() / fair_metrics.by_group['selection_rate'].min()

print("\nFairness Metrics (Reweighing):")
print(f"Statistical Parity Difference : {spd:.4f}")
print(f"Equal Opportunity Difference : {eod:.4f}")
print(f"Disparate Impact Ratio       : {di_ratio:.4f}")


Reweighed FFNN Performance (German Credit):
Accuracy : 0.7000
Precision: 0.7808
Recall   : 0.8028
F1 Score : 0.7917

Fairness Metrics (Reweighing):
Statistical Parity Difference : 0.1080
Equal Opportunity Difference : 0.3321
Disparate Impact Ratio       : 1.1501


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from fairlearn.metrics import (
    MetricFrame,
    demographic_parity_difference,
    equalized_odds_difference,
    selection_rate
)
from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.inprocessing import AdversarialDebiasing
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
import warnings
warnings.filterwarnings("ignore")

# ----------------------------
# Step 1: Load and preprocess
# ----------------------------
df = pd.read_csv(r"C:\Users\Arhamsoft\Desktop\Talha Talib Thesis\german_credit_data.csv")
df.dropna(inplace=True)

# Binary protected attribute: 1 = male, 0 = female
df["sex_binary"] = df["famges"].apply(lambda x: 0 if x == 4 else 1)

# Define target and features
y = df["kredit"]
protected = df["sex_binary"]
X = df.drop(columns=["kredit", "sex_binary"])  # exclude target and protected

# Encode categorical features
X_encoded = X.copy()
for col in X_encoded.select_dtypes(include="object").columns:
    X_encoded[col] = LabelEncoder().fit_transform(X_encoded[col])

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)
X_df = pd.DataFrame(X_scaled, columns=X_encoded.columns)

# ----------------------------
# Step 2: Create BinaryLabelDataset and Split
# ----------------------------
bld_df = pd.concat([
    X_df.reset_index(drop=True),
    y.reset_index(drop=True).rename("target"),
    protected.reset_index(drop=True).rename("sex_binary")
], axis=1)

dataset = BinaryLabelDataset(
    df=bld_df,
    label_names=["target"],
    protected_attribute_names=["sex_binary"],
    favorable_label=1,
    unfavorable_label=0
)

train_dataset, test_dataset = dataset.split([0.7], shuffle=True)

# ----------------------------------
# Step 3: Train Adversarial Debiasing
# ----------------------------------
sess = tf.Session()
ad_model = AdversarialDebiasing(
    privileged_groups=[{"sex_binary": 1}],
    unprivileged_groups=[{"sex_binary": 0}],
    scope_name='adv_debiasing_ffnn',
    sess=sess,
    debias=True,
    num_epochs=50,
    batch_size=128
)
ad_model.fit(train_dataset)

# ----------------------------------
# Step 4: Predictions and Performance
# ----------------------------------
pred_dataset = ad_model.predict(test_dataset)

y_true = test_dataset.labels.ravel()
y_pred = pred_dataset.labels.ravel()
protected_test = test_dataset.protected_attributes.ravel()

acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, zero_division=0)
rec = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)

print("Adversarial Debiasing (FFNN) Performance on German Credit:")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1 Score : {f1:.4f}")

# ----------------------------------
# Step 5: Fairness Metrics
# ----------------------------------
# Ensure lengths match before MetricFrame
assert len(y_true) == len(protected_test), "Mismatch in y_true and sensitive_features lengths"

fair_metrics = MetricFrame(
    metrics={
        "accuracy": accuracy_score,
        "precision": precision_score,
        "recall": recall_score,
        "f1": f1_score,
        "selection_rate": selection_rate
    },
    y_true=y_true,
    y_pred=y_pred,
    sensitive_features=protected_test
)

spd = demographic_parity_difference(y_true, y_pred, sensitive_features=protected_test)
eod = equalized_odds_difference(y_true, y_pred, sensitive_features=protected_test)

# Handle zero-division in Disparate Impact Ratio
sr = fair_metrics.by_group['selection_rate']
di_ratio = sr.max() / sr.min() if sr.min() > 0 else np.inf

print("\nFairness Metrics (Adversarial Debiasing):")
print(f"Statistical Parity Difference : {spd:.4f}")
print(f"Equal Opportunity Difference : {eod:.4f}")
print(f"Disparate Impact Ratio       : {di_ratio:.4f}")


pip install 'aif360[inFairness]'


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


epoch 0; iter: 0; batch classifier loss: 0.758644; batch adversarial loss: 1.068746
epoch 1; iter: 0; batch classifier loss: 0.684667; batch adversarial loss: 1.132181
epoch 2; iter: 0; batch classifier loss: 0.615493; batch adversarial loss: 1.206151
epoch 3; iter: 0; batch classifier loss: 0.629196; batch adversarial loss: 1.143985
epoch 4; iter: 0; batch classifier loss: 0.574645; batch adversarial loss: 1.276730
epoch 5; iter: 0; batch classifier loss: 0.549190; batch adversarial loss: 1.218072
epoch 6; iter: 0; batch classifier loss: 0.504350; batch adversarial loss: 1.240169
epoch 7; iter: 0; batch classifier loss: 0.501920; batch adversarial loss: 1.299249
epoch 8; iter: 0; batch classifier loss: 0.517070; batch adversarial loss: 1.300938
epoch 9; iter: 0; batch classifier loss: 0.442040; batch adversarial loss: 1.374599
epoch 10; iter: 0; batch classifier loss: 0.471912; batch adversarial loss: 1.397811
epoch 11; iter: 0; batch classifier loss: 0.483163; batch adversarial loss:

In [2]:
from aif360.algorithms.postprocessing import EqOddsPostprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from fairlearn.metrics import (
    MetricFrame, 
    demographic_parity_difference, 
    equalized_odds_difference, 
    selection_rate
)

# -----------------------------------------------------
# Step 1: Convert predicted FFNN results into dataset
# -----------------------------------------------------
# Use the same `test_dataset` and `y_pred` and `y_true` from FFNN Adversarial Debiasing output

# Create predicted dataset from FFNN output
pred_dataset = test_dataset.copy()
pred_dataset.labels = y_pred.reshape(-1, 1)

# -----------------------------------------------------
# Step 2: Equalized Odds Postprocessing
# -----------------------------------------------------
eq_odds = EqOddsPostprocessing(
    unprivileged_groups=[{'sex_binary': 0}],
    privileged_groups=[{'sex_binary': 1}],
    seed=42
)

eq_odds = eq_odds.fit(test_dataset, pred_dataset)
eq_odds_pred = eq_odds.predict(pred_dataset)

# -----------------------------------------------------
# Step 3: Performance Metrics
# -----------------------------------------------------
y_pred_eq = eq_odds_pred.labels.ravel()
protected_test = test_dataset.protected_attributes.ravel()

acc = accuracy_score(y_true, y_pred_eq)
prec = precision_score(y_true, y_pred_eq)
rec = recall_score(y_true, y_pred_eq)
f1 = f1_score(y_true, y_pred_eq)

print("Equalized Odds Postprocessing Performance on German Credit:")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1 Score : {f1:.4f}")

# -----------------------------------------------------
# Step 4: Fairness Metrics
# -----------------------------------------------------
fair_metrics = MetricFrame(
    metrics={
        "accuracy": accuracy_score,
        "precision": precision_score,
        "recall": recall_score,
        "f1": f1_score,
        "selection_rate": selection_rate
    },
    y_true=y_true,
    y_pred=y_pred_eq,
    sensitive_features=protected_test
)

spd = demographic_parity_difference(y_true, y_pred_eq, sensitive_features=protected_test)
eod = equalized_odds_difference(y_true, y_pred_eq, sensitive_features=protected_test)
di_ratio = fair_metrics.by_group['selection_rate'].max() / fair_metrics.by_group['selection_rate'].min()

print("\nFairness Metrics (Equalized Odds Postprocessing):")
print(f"Statistical Parity Difference : {spd:.4f}")
print(f"Equal Opportunity Difference : {eod:.4f}")
print(f"Disparate Impact Ratio       : {di_ratio:.4f}")


Equalized Odds Postprocessing Performance on German Credit:
Accuracy : 0.3633
Precision: 0.7586
Recall   : 0.1068
F1 Score : 0.1872

Fairness Metrics (Equalized Odds Postprocessing):
Statistical Parity Difference : 0.0205
Equal Opportunity Difference : 0.0814
Disparate Impact Ratio       : 1.2160
